In [27]:
import math
import numpy as np
import statistics
import openpyxl
import scipy.optimize as opt
import scipy.stats as stats
from statistics import mean
from scipy.integrate import quad
from scipy.optimize import root
from scipy.stats import chi2, norm, nct

# rgamma
Calculate the approxomation of the ratio of gamma(n+1)/gamma(n+3/2) for large m

In [28]:
import numpy as np

def rgamma(n):
    x = n + 1
    fval = np.sqrt(x-.25) * (1 + 1 / (64 * x * x) + 1 / (128 * x * x * x))
    return 1/fval

# ncd_cdf
Calculate the cdf of a non-central t statistic
t is the cdf variable; n is # degrees of freedom; δ is the noncentrality parameter

In [29]:
import statistics as stats
import scipy

def nct_cdf(t, n, delta, tol):
    r = lambda t_val, n_val: t_val*t_val/(t_val*t_val+n_val)
    if (t >= 0):
        out = stats.norm.cdf(-delta)
        for m in range(0, 1000):
            if (m <= 50):
                sumincr = 0.5 * stats.poisson.pmf(m, delta * delta / 2) * (stats.beta.cdf(r(t, n), m + 0.5, n / 2) + ((delta * scipy.special.gamma(m + 1)) / (np.sqrt(2) * scipy.special.gamma(m + 1.5))) * stats.beta.cdf(r(t, n), m + 1, n / 2))
            else:
                sumincr = 0.5 * stats.poisson.pmf(m, delta * delta / 2) * (stats.beta.cdf(r(t, n), m + 0.5, n / 2) + (delta / np.sqrt(2) * rgamma(m)) * stats.beta.cdf(r(t, n), m + 1, n / 2))
            out = out + sumincr

            if (sumincr / out < tol):
                break
    else:
        out = stats.norm.cdf(delta)
        for m in range(0, 1000):
            if (m <= 50):
                sumincr = 0.5 * stats.poisson.pmf(m, delta * delta / 2) * (stats.beta.cdf(r(t, n), m + 0.5, n / 2) + ((delta * scipy.special.gamma(m + 1)) / (np.sqrt(2) * scipy.special.gamma(m + 1.5))) * stats.beta.cdf(r(t, n), m + 1, n / 2))
            else:
                sumincr = 0.5 * stats.poisson.pmf(m, delta * delta / 2) * (stats.beta.cdf(r(t, n), m + 0.5, n / 2) + (delta / np.sqrt(2) * rgamma(m)) * stats.beta.cdf(r(t, n), m + 1, n / 2))

            out = out + sumincr
            if (abs(sumincr)/out < tol):
                out = 1 - out
                break
    return out

# inv_nct_cdf
Inverse of non-central t-statistic cdf; finds the threshold yielding a cdf value of P

n is the # of degrees of freedom

In [30]:
def inv_nct_cdf(P, n, delta, tol):

    # set initial t guess to lowercase delta (delta) and step up and down to find an interval where f(t) has opposite signs for the endpoints
    f = lambda t: nct_cdf(t, n, delta, tol) - P

    x0 = delta

    # Determine the direction to move in
    if f(x0) <= 0:
        x1 = 2 * delta
        while (f(x0) <= 0 and f(x1) <= 0):
            x0 = x1
            x1 = x1 + delta
    else:
        x0 = 0
        x1 = delta
        while (f(x0) > 0 and f(x1) >= 0):
            x1 = x0
            x0 = x0 - delta
    

    return opt.brentq(f, x0, x1)

# ktol
Find one-sided tolerance interval k (multiple of sample std dev away from sample mean), given γ, α and n

In [31]:
import math

def ktol(upsilon, a, n, tol):
    zp = stats.norm.ppf(1-upsilon, 0, 1)
    delta = math.sqrt(n) * zp
    result = inv_nct_cdf(1 - a, n - 1, delta, tol) / math.sqrt(n)
    return result

# round_above_threshold
A function for helping with imprecision

In [32]:
def round_above_threshold(n):
    if n - int(n) >= 0.98: 
        return round(n)
    else: 
        return n

# trap
Trapezoid of height ht, left base lb, left top lt, right top rt, right base rb

In [33]:
def trap(x, ht, lb, lt, rt, rb):
    global tr
    if x < lb:
        return 0
    if x > rb:
        return 0
    if lt <= x <= rt:
        return ht
    if lb <= x < lt:
        tr = ht * (x - lb) / (lt - lb)
    if rt < x <= rb:
        tr = ht * (rb - x) / (rb - rt)
    return tr

# bad_data
eliminates invalid intervals

In [34]:
def bad_data(x, x0, x1):
    y = None
    x = np.array(x)
    a = x[:, 0]
    b = x[:, 1]
    for i in range(0, len(x)):
        # remove rows with infeasible endpoints
        if ((x0 <= a[i] < b[i] <= x1) and (b[i] - a[i] < x1 - x0)):
            if y is None:
                y = [[a[i], b[i]]]
            else:
                y.append([a[i], b[i]])
    return y

# box_and_whisker
calculate quartiles and inter-quartile ranges on right or left interval endpoints

x is an n-vector of right or left interval endpoints

In [35]:
def box_and_whisker(x):
    # calculate quartile index
    global q75, q25
    nq = np.floor(len(x) / 4)

    # skip this test if nq = 0
    if (nq == 0):
        return x

    # find first and third quartile values
    remainder = np.mod(len(x), 4)
    y = np.sort(x)
    if (remainder == 0):
        q25 = (y[int(nq)] + y[int(nq) - 1]) / 2
        q75 = (y[int(3 * nq)] + y[int(3 * nq) - 1]) / 2
    elif (remainder == 1):
        q25 = (y[int(nq)] + y[int(nq) - 1]) / 2
        q75 = (y[int(3 * nq)] + y[int(3 * nq + 1)]) / 2
    elif (remainder == 2):
        q25 = y[int(nq)]
        q75 = y[int(3 * nq + 1)]
    elif (remainder == 3):
        q25 = y[int(nq)]
        q75 = y[int(3 * nq + 2)]

    # find inner quartile range and bounds of valid endpoints
    iqr = q75 - q25
    xmin = q25 - 1.5 * iqr
    xmax = q75 + 1.5 * iqr
    return [xmin, xmax]

# outlier_test
flags outlier endpoints

x is an n x 2 vector of right and left interval endpoints

In [36]:
def outlier_test(x):
    # calculate box and whisker test on x[0] and x[1]
    x = np.array(x)
    x0 = box_and_whisker(x[:, 0])
    x1 = box_and_whisker(x[:, 1])
    y = []

    # eliminate outlier endpoints in x using box and whisker test
    for i in range(0, len(x)):
        if (x0[0] <= x[i, 0] <= x0[1] and x1[0] <= x[i, 1] <= x1[1]):
            y.append(x[i])
    if len(y) == 0:
        return None

    y = np.array(y)
    # y has first pass outliers eliminated
    # repeat box and whisker test on L=y[1]-y[0]
    L = y[:, 1] - y[:, 0]
    Lbw = box_and_whisker(L)
    z = []
    for i in range(0, len(y)):
        if (Lbw[0] <= L[i] <= Lbw[1]):
            z.append(y[i])
    if len(z) == 0:
        return None
    return np.array(z)

# fr
Root function for Eq. A. 18 of reference [2]

In [37]:
def fr(y, a):
    r = y
    func = lambda r: norm.cdf(y + r) - norm.cdf(y - r) - (1 - a)
    return root(func, r).x[0]

# fk
Q(lambda, k) function for Eq. A. 19 of reference [2]

In [38]:
def fk(xk, a, n):
    def integrand(y):
        term1 = ((n - 1) * fr(y, a) ** 2) / (xk ** 2)
        term2 = n - 1
        pchisq = 1 - chi2.cdf(term1, term2)
        term3 = (-1 / 2) * n * y ** 2
        return ((2 * np.sqrt(n)) / (np.sqrt(2 * np.pi))) * pchisq * np.exp(term3)

    result, error = quad(integrand, 0, np.inf)
    return result

# kk
Solve for k of Eq. A. 14 of reference [2]

In [39]:
def kk(a, y, n):
    xk = 2
    func = lambda xk: fk(xk, a, n) - (1 - y)
    return root(func, xk).x[0]

# tolerance
flags valuews of x where either endpoint is outside the tolerance limits

In [40]:
def tolerance(x, m, sigma, k):
    upperlim = m + k * sigma
    lowerlim = m - k * sigma
    for i in range(len(x)):
        if lowerlim <= x[i] <= upperlim:
            continue
        else:
            x[i] = -1000
    return x

# reasonable
eliminate unreasonable intervals that do not overlap all other intervals or have too little overlap
x is an n x 2 vector of intervals
ml, mr, sigma_l, sigma_r are means and standard deviations

In [41]:
def reasonable(x, ml, mr, sigma_l, sigma_r):
    # If sigma_l or sigma_r is zero, all intervals overlap by definition
    if sigma_l == 0 or sigma_r == 0:
        return x
    # If sigma)l == sigma_r, the solution to Eq. (A-6) in Ref. [1] is the mean of the means
    if sigma_l == sigma_r:
        zeta = (ml + mr) / 2
    else:
        zeta_1 = ((mr * sigma_l ** 2 - ml * sigma_r ** 2) - sigma_l * sigma_r * np.sqrt(
            (ml - mr) ** 2 + 2 * (sigma_l ** 2 - sigma_r ** 2) * math.log(sigma_l / sigma_r))) / (
                             sigma_l ** 2 - sigma_r ** 2)
        zeta_2 = ((mr * sigma_l ** 2 - ml * sigma_r ** 2) + sigma_l * sigma_r * np.sqrt(
            (ml - mr) ** 2 + 2 * (sigma_l ** 2 - sigma_r ** 2) * math.log(sigma_l / sigma_r))) / (
                             sigma_l ** 2 - sigma_r ** 2)

        if ml <= zeta_1 <= mr:
            zeta = zeta_1
        else:
            zeta = zeta_2

    y = []

    for i in range(len(x)):

        # maybe the one line I am not proud of, because of how python handles floating point numbers, it is necessary to round the final value
        if (2 * ml - zeta <= x[i][0] < zeta < x[i][1] <= round_above_threshold(2 * mr - zeta)):
            y.append(x[i])
    return y

# dataclean helper functions
Stage 1 -- Bad data elimination

Stage 2 -- Outlier elimination

Stage 3 -- Tolerance limit processing

Stage 4 -- Reasonable interval processing

In [42]:
def eliminate_bad_data(x, x0, x1):
    y = bad_data(x, x0, x1)
    if y is None:
        return None, "All intervals eliminated at bad data stage"
    return y, None

def eliminate_outliers(y):
    z = outlier_test(y)
    if z is None:
        return None, "All intervals eliminated at outlier elimination stage"
    return z, None

def calculate_mean_std(z):
    # Calculate mean and standard deviation
    mean_left = np.mean(z[:, 0])
    std_left = statistics.stdev(z[:, 0])
    mean_right = np.mean(z[:, 1])
    std_right = statistics.stdev(z[:, 1])
    return mean_left, std_left, mean_right, std_right


def tolerance_limit_processing(z, mean_left, std_left, mean_right, std_right, a, significance_level):
    tolerance_factor = kk(a, significance_level, len(z) + 1)
    y0 = tolerance(z[:, 0], mean_left, std_left, tolerance_factor)
    y1 = tolerance(z[:, 1], mean_right, std_right, tolerance_factor)
    z = []

    for i in range(len(y0)):
        if int(y0[i]) != -1000 and int(y1[i]) != -1000:
            z.append([y0[i], y1[i]])

    if z:
        return np.array(z), None
    else:
        return np.empty((0, 2)), "All intervals eliminated at tolerance limit processing stage"


def reasonable_interval_processing(z, mean_left, std_left, mean_right, std_right):
    z = reasonable(z.tolist(), mean_left, mean_right, std_left, std_right)
    if z:
        return np.array(z), None
    else:
        return np.empty((0, 2)), "All intervals eliminated at reasonable interval processing stage"

# dataclean
Preprocess raw interval data for a given word to eliminate unacceptable intervals using all of the above tests

In [43]:
def dataclean(x, x0, x1, a, significance_level):
    y = bad_data(x, x0, x1)
    if y is None:
        return "All intervals eliminated at bad data stage"

    z, message = eliminate_outliers(y)
    if message:
        return message

    mean_left, std_left, mean_right, std_right = calculate_mean_std(z)

    z, message = tolerance_limit_processing(z, mean_left, std_left, mean_right, std_right, a, significance_level)
    if message:
        return message

    z, message = reasonable_interval_processing(z, mean_left, std_left, mean_right, std_right)
    if message:
        return message

    out = [[], [], []]
    out[0] = z
    # Compute sample means of residual interval endpoints
    out[1] = np.mean(z[:, 0])
    out[2] = np.mean(z[:, 1])
    return out

# hma_fou_class0
Step 1 of HMA: determine the FOU class for naturally bounded (class 0) sets

x is an n x 2 array of intervals that survived the dataclean process

In [44]:
def hma_fou_class0(x, x0, x1):
    # [x0, x1] is the bound interval
    ml = np.mean(x[:, 0])
    sigma_l = statistics.stdev(x[:, 0])
    mr = np.mean(x[:, 1])
    sigma_r = statistics.stdev(x[:, 1])
    k = nct_cdf.ktol(0.05, 0.05, len(x), 10 ** -5)
    al = ml - k * sigma_l
    bu = mr + k * sigma_r
    if al <= x0:
        out = "Left shoulder FOU"
    else:
        if bu >= x1:
            out = "Right shoulder FOU"
        else:
            out = "Interior FOU"
    return out

# hma_fou_class1
Step 1 of HMA: determine the FOU class for sets bounded only on the left (class 1)

In these cases, there are only Left shoulder or Interior FOUs

x is an n x 2 array of intervals that survived the dataclean processing

In [45]:
def hma_fou_class1(x, x0):
    # x0 is the left bound (typically 0) for the intervals
    ml = np.mean(x[:, 0])
    sigma_l = statistics.stdev(x[:, 0])
    k = nct_cdf.ktol(0.05, 0.05, len(x), 10 ** -5)
    al = ml - k * sigma_l
    if al <= x0:
        out = "Left shoulder FOU"
    else:
        out = "Interior FOU"
    return out

# hma_overlap
Compute overlap interval of x rows for FOU class c

Note: the data part eliminates non-overlapping intervals, so we are assured of overlap

In [46]:
def hma_overlap(x, c, x0, x1):
    if c == "Left shoulder FOU":
        out = [x0, min(x[:, 1])]
    elif c == "Interior FOU":
        out = [max(x[:, 0]), min(x[:, 1])]
    elif c == "Right shoulder FOU":
        out = [max(x[:, 0]), x1]
    return out

# hma_olap_remove
Given interval array x and FOU class, remove overlap from original intervals

For Left or Right shoulder FOUs, this leaves a single array of smaller intervals

For Interior FOUs, this leaves a nested 2-vector of arrays of smaller intervals for the left/right sections of the FOU, respectively

In [47]:
def hma_olap_remove(x, c):
    out = []
    if c == "Left shoulder FOU":
        bmin = min(x[:, 1])
        for i in range(len(x)):
            out.append([bmin, x[i, 1]])
    elif c == "Right shoulder FOU":
        amax = max(x[:, 0])
        for i in range(len(x)):
            out.append([x[i, 0], amax])
    elif c == "Interior FOU":
        out0 = []
        out1 = []
        amax = max(x[:, 0])
        bmin = min(x[:, 1])
        for i in range(len(x)):
            out0.append([x[i, 0], amax])
            out1.append([bmin, x[i, 1]])
        out = [out0, out1]
    return out

# aleft
Calculate a parameters for left-hand side of Interior or Right shoulder FOU

See eq. (5) in HMA paper

xr is the set of reduced intervals for the left-hand side from the hmaolapremove function
    
x0 is the left bound
    
oleft is the left bound of the overlap interval of the original interval set
    
Thus, oleft will be the max right bound of the reduced intervals xr

In [48]:
def aleft(xr, x0):
    xr = np.array(xr)
    oleft = max(xr[:, 0])
    intlengths = np.abs(xr[:, 0] - oleft)
    # mLH is the mean of interval lengths wrt the left bound of the overlap interval (oleft)
    mLH = oleft - np.mean(intlengths)
    sLH = statistics.stdev(intlengths)
    a_left = max(x0, oleft - 3 * np.sqrt(2) * sLH)
    a_right = min(oleft, 6 * mLH + 3 * np.sqrt(2) * sLH - 5 * oleft)
    # Now test to if the order is sensible, and if not, reverse them
    if a_left <= a_right:
        out = [a_left, a_right]
    else:
        out = [max(x0, a_right), min(oleft, a_left)]
    # out_0/out_1 are the UMF/LMF intersections with the x-axis, respectively
    return out

# aright
Calculate a parameters for right-hand side of Interior or Left shoulder FOU

See eq. (6) in HMA paper

xr is the set of reduced intervals for the right-hand side from the hmaolapremove function

x1 is the right bound

oright is the right bound of the overlap interval of the original interval set

Thus, oright will be the min left bound of the reduced intervals xr

In [49]:
def aright(xr, x1):
    xr = np.array(xr)
    oright = min(xr[:, 1])
    intlengths = np.abs(xr[:, 1] - oright)
    #mRH is the mean of interval length wrt the right bound of the overlap interval (oright)
    mRH = oright + np.mean(intlengths)
    sRH = statistics.stdev(intlengths)
    b_right = min(x1, oright + 3 * np.sqrt(2) * sRH)
    b_left = max(oright, 6 * mRH - 3 * np.sqrt(2) * sRH - 5 * oright)
    # Now test to if the order is sensible, and if not, reverse them
    if b_left <= b_right:
        out = [b_left, b_right]
    else:
        out = [max(oright, b_right), min(x1, b_left)]
    # out_0/out_1 are the UMF/LMF intersections with the x-axis, respectively
    return out

# hma_map
Implement the mapping of fuzzy part step 4

x is the set of reduced intervals from the hmaolapremove function

In [50]:
def hma_map(x, c, x0, x1):
    """Implement the mapping of fuzzy part step 4
    x is the set of reduced intervals from the hmaolapremove function"""
    if c == "Interior FOU":
        out = [aleft(x[0], x0), aright(x[1], x1)]
    elif c == "Left shoulder FOU":
        out = [aright(x, x1)]
    elif c == "Right shoulder FOU":
        out = [aleft(x, x0)]
        # out will be a single 2-vector for left or right-shoulder FOU, a nested 2-vector of 2-vectors for interior FOUs
    return out

# hma_trap
Go from an array of intervals x to the UMF/LMF trapezoidal parameters

In [51]:
def hma_trap(x, x0, x1):
    # Go from an array of intervals x to the UMF/LMF trapezoidal parameters
    c = hma_fou_class0(x, x0, x1)
    # compute overlap interval
    olap_interval = hma_overlap(x, c, x0, x1)
    # compute reduced intervals with overlap removed
    x_reduced = hma_olap_remove(x, c)
    # compute the trapezoidal parameters
    tp = hma_map(x_reduced, c, x0, x1)
    # calculate trapezoid parameters for different FOUs
    if c == "Interior FOU":
        # tp is a nested 2-vector of 2-vectors for left/right UMF/LMF x-axis intercepts
        out_UMF = [tp[0][0], olap_interval[0], olap_interval[1], tp[1][1]]
        out_LMF = [tp[0][1], olap_interval[0], olap_interval[1], tp[1][0]] 
    elif c == "Left shoulder FOU":
        # tp is a 2-vector of UMF/LMF x-axis intercepts
        out_UMF = [x0, x0, olap_interval[1], tp[0][1]]
        out_LMF = [x0, x0, olap_interval[1], tp[0][0]]
    elif c == "Right shoulder FOU":
        # tp is a 2-vector of UMF/LMF x-axis intercepts
        out_UMF = [tp[0][0], olap_interval[0], x1, x1]
        out_LMF = [tp[0][1], olap_interval[0], x1, x1]
    return [out_UMF, out_LMF]

# trap_z
Trapzoidal function with h parameters

In [52]:
def trap_z(x, h):
    """Trapzoidal function with h parameters"""
    return [noncenteral_tstatic_cdf.trapz(x, 1, h[0][0], h[1][0], h[2][0], h[3][0]), noncenteral_tstatic_cdf.trapz(x, 1, h[0][1], h[1][1], h[2][1], h[3][1])]